# NEXUS tool: case study for the Jordan - energy demand calculations
In this notebook a case study for the Jordan country is covered using the `nexustool` package. The water requirements for agricultural irrigation, residential, industrial and tourism use were previously calculated using the Water Evaluation and Planning System (WEAP) model. In this case study, the energy requirements for groundwater pumping, wastewater treatment, desalination of seawater and pumping for water conveyance are estimated.

First import the package by running the following block:

In [ ]:
import sys
sys.path.append("..") #this is to add the avobe folder to the package directory
import os
import nexustool
import pandas as pd

## 1. Read scenario data
After importing all required packages, the input GIS data is loaded into the variable `df`. Change the `data_folder`, `scenario` and `climate` variables to reflect the name and relative location of your data file. This dataset should already have the water demand for irrigation data.

In [ ]:
data_folder = os.path.join('data', 'processed results')
scenario = 'Reference'
climate = 'Climate Change'
input_folder = os.path.join(data_folder, scenario, climate)

## 2. Create nexus model 
To create a model simply create an instance of the `nexustool.Model()` class and store it in a variable name. The `nexustool.Model()` class requires a dataframe as input data. Several other properties and parameter values can be defined by explicitly passing values to them. To see a full list of parameters and their explaination refer to the documentation of the package. We wil create a model using the `pipelines_flow.gz` data:

In [ ]:
file_path = os.path.join(input_folder, 'pipelines_flow.gz')
df = pd.read_csv(file_path)
jordan = nexustool.Model(df)

## 3. Define variable names
The names of the properties of the model can be changed at any time. This is important for the model to know how each property is called withing your input data. To check the current property names run the `.print_properties()` method, a list with the names of each property and its current value will be displayed.

Then you can provide the right names for each property, calling them and assigning a value as:
```python
jordan.elevation = 'elevation_delta'
jordan.gw_depth = 'name_of_ground_water_depth'
```

In this particular case we will need to change the following default values:

In [ ]:
jordan.elevation_diff = 'elevation_delta' # for the case of GW, the elevation_diff is set to be the wtd
jordan.L = 'segment_length' # for the case of GW, the distance is set to be the wtd
jordan.D = 'Pipe_diameter'

# Defines the name of the variable for Peak Water Demand and Seasonal Scheme Water demand (monthly)
jordan.sswd = 'sswd' # Seassonal Scheme Water Demand
jordan.pwd = 'pwd' # Peak Water Demand
jordan.pp_e = 'pp_e' # Peak Pumping Energy
jordan.pa_e = 'pa_e' # Pumping Average Energy

## 4. Define pipelines diameters and pumping efficiency
Now we need to define the specifications of the water network, giving pipeline / canal diameter values and surface pumping efficiency:

In [ ]:
jordan.df['Pipe_diameter'] = 2 #in m
jordan.df.loc[(df['pipeline']=='KAC'), 'Pipe_diameter'] = 4 #in m
jordan.df.loc[(df['pipeline']=='PL_Disi'), 'Pipe_diameter'] = 1.4 #in m
jordan.df.loc[(df['pipeline']=='PL_RedDead'), 'Pipe_diameter'] = 1.4 #in m
jordan.df.loc[(df['pipeline']=='PS_ZaraMain'), 'Pipe_diameter'] = 1.8 #in m
jordan.df.loc[(df['pipeline']=='PL_KACToZay'), 'Pipe_diameter'] = 1.4 #in m
jordan.df.loc[(df['pipeline']=='PL_Dab_AinGhazal'), 'Pipe_diameter'] = 1.4 #in m
jordan.df.loc[(df['pipeline']=='PL_ZaytoDabouq'), 'Pipe_diameter'] = 1.4 #in m

jordan.pumping_hours_per_day = 10
jordan.pump_eff = 1 # pumping efficiency

## 5. Peak Water Demand (PWD)
The $PWD$ is definfe as the daily peak cubic meters of water pumped per second withing the month. To accomplish that, the $SSWD$ (m<sup>3</sup>/month) is divided by 30 days per month, 3600 seconds per hour and the amount of average pumping hours in a day. This provides the $PWD$ in m<sup>3</sup>/s:

$$
PWD\,(m^3/s) = \frac{SSWD\,(m^3/month)}{30\,(day/month)\cdot PumpHours\,(h/day)\cdot 3600\, (s/h)}
$$

Moreover, the $PWD$ for agricultural irrigation is assumed as double the normal $PWD$. We make this calculations as per the following cell:

In [ ]:
#Defines the PWD. It is defined as double the seasonal demand for agricultural sites
jordan.df[jordan.pwd] = jordan.df[jordan.sswd] / 30 / jordan.pumping_hours_per_day / 3600 #to convert to cubic meter per second [m3/s]
jordan.df.loc[jordan.df['type']=='Agriculture', jordan.pwd] *= 2

## 6. Calculate pumping energy requirements
To estimate the pumping energy requirements for conveyance, first we need to calculate the Total Dinamic Head (TDH). This, is a measure in meters that accounts for the elevation difference between two points and the pressure loss in distribution.

For that, the area $A$ `.pipe_area()`, the velocity $V$ `.flow_velocity()`, the Reynolds number $Re$ `.reynolds()` and the friction factor $f$ `.friction_factor()` need to be estimated. The `nexustool` provides simple functions that allows us make an easy estimation of these variables, which have the following formulas implemented in the background:

$$
A\,(m^2) = \pi\cdot \frac{D^2}{4}
$$

$$
V\,(m/s) = \frac{SSWD\,(m^3/month)}{PumpHours\,(h/day)\cdot 30\,(day/month)\cdot 3600\,(s/h)\cdot A\,(m^2)}
$$

$$
Re = \frac{V\,(m/s)\cdot D\,(m)}{v\,(m^2/s)}
$$

Where $v$ is the kinematic viscosity of water at around 1.004e-06 m<sup>2</sup>/s. And the frction factor is estimated according to the Swamee–Jain equation:

$$
f = \frac{0.25}{\left[log_{10}\left(\frac{\epsilon}{3.7D}+\frac{5.74}{Re^{0.9}}\right)\right]^2}
$$

Where $\epsilon$ is the roughness of the material. 

In [ ]:
jordan.pipe_area()
jordan.flow_velocity()
jordan.reynolds()
jordan.friction_factor()

Then, the TDH can be calculated by simply calling the `.get_tdh()` function.

$$
TDH\,(m) = \Delta H + f\cdot \frac{L\,(m)}{D\,(m)}\cdot \frac{V(m/s)^2}{2\cdot g\,(m/s^2)}
$$

Whereas the conveyance pumping energy requirements by calling the `.get_pumping_energy()` method. The equation used to calculate the Electricity Demand ($E_D$) for pumping is as follows:

$$
E_D\,(kW_h) = \frac{SSWD\,(m^3)\cdot \rho\,(kg/m^3)\cdot g\,(m/s^2)\cdot TDH\,(m)}{PP_{eff}\,(\%)\cdot 3600\,(s/h)\cdot 1000\,(W/kW)}
$$

The variable withing the Model for the $E_D$ is the `pa_e` or Pumping Average Electricity requirements.

Moreover, the Power Demand for pumping ($PD$) is denoted by the variable `pp_e` and calculated by the following formula:

$$
PD\,(kW) = \frac{PWD\,(m^3/s)\cdot \rho\,(kg/m^3)\cdot g\,(m/s^2)\cdot TDH\,(m)}{PP_{eff}\,(\%)\cdot 1000\,(W/kW)}
$$

The `.get_pumping_energy()` method calculates both the $E_D$ (`pa_e`) and $PD$ (`pp_e`).

In [ ]:
jordan.get_tdh(friction=False)
jordan.get_pumping_energy()

jordan.df.loc[(jordan.df['pipeline']=='KAC'), 'pa_e'] = 0 #Setting energy for canal to 0

In [ ]:
jordan.df.loc[(jordan.df['Year']==2020), 'pa_e'].sum() / jordan.df.loc[(jordan.df['Year']==2020), 'water_use'].sum()

In [ ]:
jordan_gw.df.loc[(jordan_gw.df['Year']==2020), 'sswd'].sum()

In [ ]:
jordan_gw.df.loc[(jordan_gw.df['Year']==2020) & (jordan_gw.df['point'].str.contains('Desert')), 'pa_e'].sum() / \
jordan_gw.df.loc[(jordan_gw.df['Year']==2020) & (jordan_gw.df['point'].str.contains('Desert')), 'sswd'].sum()

In [ ]:
import geopandas as gpd
pipelines = gpd.read_file(os.path.join('data', 'gis', 'processed layers', 'pipelines.gpkg'))
pipelines.rename(columns={'index': 'pl_index'}, inplace=True)
pipelines = pipelines.merge(jordan.df.loc[(jordan.df['Year']==2020)].groupby('pipeline').agg({'pa_e': 'sum', 'sswd': 'sum'}), on='pipeline')

In [ ]:
pipelines.plot(column='sswd', figsize=(16, 16), legend=True)

## 7. Create nexus model for groundwater pumping
The steps needed to estimate the groundwater pumping energy requirements are farily similar to the previous ones. We need to set the path to the `groundwater_supply.csv`, read it into a dataframe and create a nexus Model with it. Then, we define the pipe diameter and some required atttributes names:

In [ ]:
file_path = os.path.join(input_folder, 'groundwater_supply.gz')
df_groundwater = pd.read_csv(file_path)

jordan_gw = nexustool.Model(df_groundwater)

jordan_gw.df['Pipe_diameter'] = 1
jordan_gw.elevation_diff = 'wtd_m'
jordan_gw.L = 'wtd_m'

# Defines the name of the variable for Peak Water Demand and Seasonal Scheme Water demand (monthly)
jordan_gw.sswd = 'sswd' # Seassonal Scheme Water Demand
jordan_gw.pwd = 'pwd' # Peak Water Demand
jordan_gw.pp_e = 'pp_e' # Peak Pumping Energy
jordan_gw.pa_e = 'pa_e' # Pumping Average Energy

In [ ]:
#Defines the PWD. It is defined as double the seasonal demand for agricultural sites
jordan_gw.df[jordan_gw.pwd] = jordan_gw.df[jordan_gw.sswd] / 30 / jordan_gw.pumping_hours_per_day / 3600 #to convert to cubic meter per second [m3/s]
jordan_gw.df.loc[jordan_gw.df['type']=='Agriculture', jordan_gw.pwd] *= 2

## 8. Calcuculate groundwater pumping energy
The energy requirements are then calculated calling the `.pipe_area()`, `.flow_velocity()`, `.reynolds()`, `.friction_factor()`, `.get_tdh()` and get `.get_pumping_energy()` methods.

In [ ]:
jordan_gw.pipe_area()
jordan_gw.flow_velocity()
jordan_gw.reynolds()
jordan_gw.friction_factor()

jordan_gw.get_tdh(friction=False)
jordan_gw.get_pumping_energy()

## 9. Calculate wastewater treatment energy
To estimate the energy requirements for wastewater treatment, we read the `wwtp_inflow.csv` data into a dataframe and multiply the `value` attribute, wich represents the amount of treated water, by a defined energy intensity representative of the treatment process.

In [ ]:
file_path = os.path.join(input_folder, 'wwtp_inflow.gz')
df_wwtp = pd.read_csv(file_path)

wwtp_energy_int = 0.6 # kWh/m3
df_wwtp['pa_e'] = df_wwtp['sswd'] * wwtp_energy_int

## 10. Calculate desalination energy
Finally, we need to calculate the energy requirements for desalination. Thus we read the `desalination.gz` data into a dataframe and similarly to the wastewater treatment case, we multiply the desalinated water `sswd` by each desalination plant, by a respective energy intensity value.

In [ ]:
file_path = os.path.join(input_folder, 'desalination.gz')
df_desal = pd.read_csv(file_path)

red_dead_energy_int = 3.31 # kWh/m3
aqaba_energy_int = 5 # kWh/m3
df_desal['pa_e'] = 0

df_desal.loc[df_desal.point=='RedDead', 'pa_e'] = df_desal.loc[df_desal.point=='RedDead', 'sswd'] * red_dead_energy_int
df_desal.loc[df_desal.point=='Aqaba Desal', 'pa_e'] = df_desal.loc[df_desal.point=='Aqaba Desal', 'sswd'] * aqaba_energy_int

## 11. Save result files
The results can then be saved into a defined output folder `results_folder`:

In [ ]:
results_folder = os.path.join('dashboard', 'data', scenario, climate)
os.makedirs(results_folder, exist_ok=True)

jordan.df.to_csv(os.path.join(results_folder, 'pipelines_data.gz'), index=False)
jordan_gw.df.to_csv(os.path.join(results_folder, 'groundwater_pumping.gz'), index=False)
df_wwtp.to_csv(os.path.join(results_folder, 'wwtp_data.gz'), index=False)
df_desal.to_csv(os.path.join(results_folder, 'desal_data.gz'), index=False)